In [2]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from PIL import Image
import os
import pandas as pd
import timm
from sklearn.model_selection import train_test_split
from torch.optim import AdamW
from torch.optim.lr_scheduler import CosineAnnealingLR
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Read the training data
train_df = pd.read_csv('train.csv')

# Define the class mapping
class_mapping = {
    0: "Healthy",
    1: "Common Rust",
    2: "Blight",
    3: "Gray Leaf Spot"
}

# Display basic information
print("Dataset Info:")
print(train_df.info())
print("\nFirst few rows:")
print(train_df.head())
print("\nClass distribution:")
print(train_df['Label'].value_counts())

# Visualize class distribution
plt.figure(figsize=(10, 6))
sns.countplot(data=train_df, x='Label')
plt.title('Distribution of Leaf Disease Classes')
plt.xlabel('Disease Class')
plt.ylabel('Number of Images')
plt.xticks(range(4), [class_mapping[i] for i in range(4)], rotation=45)
plt.tight_layout()
plt.show()

# Check image directory
train_path = 'data/train'
image_files = os.listdir(train_path)
print(f"\nNumber of images in {train_path}: {len(image_files)}")
print("Sample image names:", sorted(image_files)[:5])

# Load and display a sample image
sample_img_path = os.path.join(train_path, image_files[0])
sample_img = Image.open(sample_img_path)
plt.figure(figsize=(6, 6))
plt.imshow(sample_img)
plt.title(f"Sample Image\nSize: {sample_img.size}")
plt.axis('off')
plt.show()

In [ ]:
# Visualize class distribution with proper labels
plt.figure(figsize=(10, 6))
class_counts = train_df['Label'].value_counts()

# Create bar plot
sns.barplot(x=class_counts.index.map(class_mapping), y=class_counts.values)

# Customize plot
plt.title('Distribution of Maize Leaf Disease Classes', fontsize=12, pad=20)
plt.xlabel('Disease Class')
plt.ylabel('Number of Images')
plt.xticks(rotation=45)

# Add value labels on top of each bar
for i, v in enumerate(class_counts.values):
    plt.text(i, v, str(v), ha='center', va='bottom')

plt.tight_layout()
plt.show()

# Print percentage distribution
print("\nClass Distribution:")
for label, count in class_counts.items():
    percentage = (count/len(train_df))*100
    print(f"{class_mapping[label]}: {count} images ({percentage:.2f}%)")

In [ ]:
class MaizeLeafDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None):
        self.leaf_data = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.transform = transform
        
    def __len__(self):
        return len(self.leaf_data)
    
    def __getitem__(self, idx):
        img_name = f"{self.leaf_data.iloc[idx]['Image']}.jpg"
        img_path = os.path.join(self.img_dir, img_name)
        
        # Load and convert image to RGB
        image = Image.open(img_path).convert('RGB')
        label = self.leaf_data.iloc[idx]['Label']
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

# Test the dataset
def test_dataset():
    # Simple transform for testing
    test_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor()
    ])
    
    # Create a test dataset
    test_dataset = MaizeLeafDataset(
        csv_file='train.csv',
        img_dir='data/train',
        transform=test_transform
    )
    
    # Get a sample
    image, label = test_dataset[0]
    
    print(f"Image tensor shape: {image.shape}")
    print(f"Label: {label} ({class_mapping[label]})")
    
    # Visualize the transformed image
    plt.figure(figsize=(6, 6))
    plt.imshow(image.permute(1, 2, 0))  # Convert from CxHxW to HxWxC for display
    plt.title(f"Transformed Image\nLabel: {class_mapping[label]}")
    plt.axis('off')
    plt.show()

# Run the test
test_dataset()

In [ ]:
# Define transforms for training and validation
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                       std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                       std=[0.229, 0.224, 0.225])
])

# Create full dataset
full_dataset = MaizeLeafDataset(
    csv_file='train.csv',
    img_dir='data/train',
    transform=train_transform
)

# Split into train and validation
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(
    full_dataset, [train_size, val_size]
)

# Create data loaders
train_loader = DataLoader(
    train_dataset,
    batch_size=16,
    shuffle=True,
    num_workers=0,
    pin_memory=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=16,
    shuffle=False,
    num_workers=0,
    pin_memory=True
)

# Test the data loaders
def test_data_loaders():
    # Get a batch of training data
    images, labels = next(iter(train_loader))
    
    print("Training Data:")
    print(f"Batch image shape: {images.shape}")
    print(f"Batch label shape: {labels.shape}")
    print(f"Sample labels: {labels[:5]}")
    print(f"Sample labels mapped: {[class_mapping[label.item()] for label in labels[:5]]}")
    
    # Display a grid of images from the batch
    plt.figure(figsize=(15, 7))
    grid_size = min(4, images.size(0))
    for i in range(grid_size):
        plt.subplot(1, grid_size, i + 1)
        # Denormalize the images for display
        img = images[i].permute(1, 2, 0)
        img = img * torch.tensor([0.229, 0.224, 0.225]) + torch.tensor([0.485, 0.456, 0.406])
        img = torch.clamp(img, 0, 1)
        plt.imshow(img)
        plt.title(f"{class_mapping[labels[i].item()]}")
        plt.axis('off')
    plt.tight_layout()
    plt.show()
    
    print("\nDataloader sizes:")
    print(f"Training batches: {len(train_loader)}")
    print(f"Validation batches: {len(val_loader)}")
    print(f"Training samples: {len(train_dataset)}")
    print(f"Validation samples: {len(val_dataset)}")

# Run the test
test_data_loaders()

In [10]:
def set_seeds(seed=42):
    """
    Set random seeds for reproducibility
    """
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


set_seeds(42)


train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(
    full_dataset, 
    [train_size, val_size],
    generator=torch.Generator().manual_seed(42)  # Add fixed seed
)


def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)

g = torch.Generator()
g.manual_seed(42)

train_loader = DataLoader(
    train_dataset,
    batch_size=16,
    shuffle=True,
    num_workers=0,
    pin_memory=True,
    worker_init_fn=seed_worker,
    generator=g
)

val_loader = DataLoader(
    val_dataset,
    batch_size=16,
    shuffle=False,
    num_workers=0,
    pin_memory=True,
    worker_init_fn=seed_worker,
    generator=g
)

In [ ]:
class SwinMaizeClassifier(nn.Module):
    def __init__(self, num_classes=4, model_name='swin_base_patch4_window7_224'):
        super().__init__()
        
        # Load base model
        self.model = timm.create_model(
            model_name,
            pretrained=True,
            num_classes=0,
            global_pool=''
        )
        
        # Get number of features
        n_features = self.model.num_features
        
        # Create custom classifier head
        self.classifier = nn.Sequential(
            nn.Linear(n_features, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, num_classes)
        )
    
    def forward(self, x):
        # Get features from Swin
        x = self.model.forward_features(x)  # [B, H, W, C]
        print(f"Shape after forward_features: {x.shape}")
        
        # Global average pooling over both spatial dimensions
        x = x.mean(dim=[1, 2])  # Take mean over both H and W dimensions
        print(f"Shape after mean pooling: {x.shape}")
        
        # Classification
        x = self.classifier(x)
        print(f"Shape after classifier: {x.shape}")
        
        return x

# Test the model
def test_fixed_model():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")
    
    # Create model instance
    model = SwinMaizeClassifier().to(device)
    model.eval()
    
    # Get a batch of data
    images, labels = next(iter(train_loader))
    images = images.to(device)
    
    print(f"\nInput image shape: {images.shape}")
    
    # Do a forward pass
    with torch.no_grad():
        outputs = model(images)
    
    print(f"\nFinal output shape: {outputs.shape}")
    print(f"Labels shape: {labels.shape}")
    
    # Test if output matches expected shape
    assert outputs.shape == torch.Size([images.shape[0], 4]), \
        f"Expected output shape [{images.shape[0]}, 4] but got {outputs.shape}"
    
    # Print sample predictions
    print("\nSample predictions:")
    probs = torch.softmax(outputs, dim=1)
    _, predicted = torch.max(outputs.data, 1)
    print(f"Predictions: {predicted[:5]}")
    print(f"Actual labels: {labels[:5]}")
    
    # Print prediction probabilities
    print("\nPrediction probabilities for first sample:")
    print(probs[0])
    
    return model

# Run the test
model = test_fixed_model()

In [ ]:
from sklearn.metrics import f1_score
import numpy as np

def train_model(model, train_loader, val_loader, num_epochs=10):
    device = next(model.parameters()).device
    criterion = nn.CrossEntropyLoss()
    
    optimizer = AdamW(
        model.parameters(),
        lr=1e-4,
        weight_decay=0.01,
        betas=(0.9, 0.999)
    )
    
    scheduler = CosineAnnealingLR(
        optimizer,
        T_max=num_epochs,
        eta_min=1e-6
    )
    
    # Training metrics
    best_val_f1 = 0.0
    train_losses = []
    val_accuracies = []
    val_f1_scores = []
    
    for epoch in range(num_epochs):
        # Training phase
        model.train()
        running_loss = 0.0
        train_predictions = []
        train_labels = []
        
        for batch_idx, (images, labels) in enumerate(train_loader):
            images, labels = images.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            
            optimizer.step()
            
            running_loss += loss.item()
            
            # Collect predictions for F1 score
            _, predicted = torch.max(outputs.data, 1)
            train_predictions.extend(predicted.cpu().numpy())
            train_labels.extend(labels.cpu().numpy())
            
            if batch_idx % 20 == 0:
                print(f'Epoch [{epoch+1}/{num_epochs}], '
                      f'Step [{batch_idx}/{len(train_loader)}], '
                      f'Loss: {loss.item():.4f}')
        
        # Calculate training F1 score
        train_f1 = f1_score(
            np.array(train_labels), 
            np.array(train_predictions), 
            average='weighted'
        ) * 100
        
        # Validation phase
        model.eval()
        correct = 0
        total = 0
        val_loss = 0.0
        val_predictions = []
        val_labels = []
        
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
                
                # Collect predictions for F1 score
                val_predictions.extend(predicted.cpu().numpy())
                val_labels.extend(labels.cpu().numpy())
        
        # Calculate metrics
        epoch_loss = running_loss / len(train_loader)
        epoch_val_loss = val_loss / len(val_loader)
        val_accuracy = 100 * correct / total
        val_f1 = f1_score(
            np.array(val_labels), 
            np.array(val_predictions), 
            average='weighted'
        ) * 100
        
        train_losses.append(epoch_loss)
        val_accuracies.append(val_accuracy)
        val_f1_scores.append(val_f1)
        
        print(f'\nEpoch [{epoch+1}/{num_epochs}]:')
        print(f'Training Loss: {epoch_loss:.4f}')
        print(f'Training F1 Score: {train_f1:.2f}%')
        print(f'Validation Loss: {epoch_val_loss:.4f}')
        print(f'Validation Accuracy: {val_accuracy:.2f}%')
        print(f'Validation F1 Score: {val_f1:.2f}%')
        
        # Calculate per-class F1 scores for validation set
        per_class_f1 = f1_score(
            np.array(val_labels), 
            np.array(val_predictions), 
            average=None
        ) * 100
        
        print("\nPer-class Validation F1 Scores:")
        for i, class_f1 in enumerate(per_class_f1):
            print(f"{class_mapping[i]}: {class_f1:.2f}%")
        
        # Save best model based on F1 score
        if val_f1 > best_val_f1:
            best_val_f1 = val_f1
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'val_accuracy': val_accuracy,
                'val_f1_score': val_f1,
                'per_class_f1': per_class_f1.tolist(),
            }, 'best_swin_model.pth')
            print(f'\nBest model saved with F1 Score: {best_val_f1:.2f}%')
        
        scheduler.step()
    
    # Plot training curves
    plt.figure(figsize=(15, 5))
    
    plt.subplot(1, 3, 1)
    plt.plot(train_losses)
    plt.title('Training Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    
    plt.subplot(1, 3, 2)
    plt.plot(val_accuracies)
    plt.title('Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy (%)')
    
    plt.subplot(1, 3, 3)
    plt.plot(val_f1_scores)
    plt.title('Validation F1 Score')
    plt.xlabel('Epoch')
    plt.ylabel('F1 Score (%)')
    
    plt.tight_layout()
    plt.show()
    
    return train_losses, val_accuracies, val_f1_scores

# Start training
train_losses, val_accuracies, val_f1_scores = train_model(model, train_loader, val_loader, num_epochs=20)

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt


def plot_raw_confusion_matrix(model, val_loader, device):
    """
    Generate and plot raw confusion matrix showing actual counts
    """
    # Set model to evaluation mode
    model.eval()
    
    # Initialize lists to store predictions and true labels
    all_predictions = []
    all_labels = []
    
    # Class mapping for labels
    class_mapping = {
        0: "Healthy",
        1: "Common Rust",
        2: "Blight",
        3: "Gray Leaf Spot"
    }
    
    # Collect predictions
    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            
            all_predictions.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    # Calculate confusion matrix
    cm = confusion_matrix(all_labels, all_predictions)
    
    # Create figure
    plt.figure(figsize=(10, 8))
    
    # Create heatmap
    sns.heatmap(cm, 
                annot=True,           # Show numbers in cells
                fmt='d',              # Use integer format
                cmap='Blues',         # Use Blues colormap
                xticklabels=[class_mapping[i] for i in range(4)],
                yticklabels=[class_mapping[i] for i in range(4)])
    
    plt.title('Confusion Matrix (Raw Counts)')
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.xticks(rotation=45, ha='right')
    plt.yticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()
    
    # Print actual confusion matrix values
    print("\nConfusion Matrix (Raw Counts):")
    print("\nTrue Labels (rows) vs Predicted Labels (columns):")
    print("\nPredicted:")
    print("         ", end="")
    for i in range(4):
        print(f"{class_mapping[i]:<15}", end="")
    print("\nActual")
    for i in range(4):
        print(f"{class_mapping[i]:<9}", end="")
        for j in range(4):
            print(f"{cm[i,j]:<15}", end="")
        print()

# Load and run confusion matrix analysis
def run_confusion_matrix_analysis():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = SwinMaizeClassifier().to(device)
    checkpoint = torch.load('best_swin_model.pth')
    model.load_state_dict(checkpoint['model_state_dict'])
    
    plot_raw_confusion_matrix(model, val_loader, device)

# Run the analysis
run_confusion_matrix_analysis()

In [ ]:
def predict_test_set(model_path='best_swin_model.pth'):
    # Set up device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")
    
    # Load the best model
    model = SwinMaizeClassifier().to(device)
    checkpoint = torch.load(model_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()
    
    # Test transform (same as validation, no augmentation)
    test_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                           std=[0.229, 0.224, 0.225])
    ])
    
    # Get test files
    test_files = sorted(os.listdir('data/test'), 
                       key=lambda x: int(x.split('.')[0]))
    predictions = []
    confidence_scores = []
    
    print(f"Processing {len(test_files)} test images...")
    
    with torch.no_grad():
        for img_name in tqdm(test_files, desc="Predicting"):
            # Load and preprocess image
            img_path = os.path.join('data/test', img_name)
            image = Image.open(img_path).convert('RGB')
            image = test_transform(image).unsqueeze(0).to(device)
            
            # Make prediction
            outputs = model(image)
            probs = torch.softmax(outputs, dim=1)
            confidence, predicted = torch.max(probs, 1)
            
            predictions.append(predicted.item())
            confidence_scores.append(confidence.item())
    
    # Create submission DataFrame
    submission_df = pd.DataFrame({
        'Image': range(len(predictions)),
        'Label': predictions
    })
    
    # Print prediction distribution
    print("\nPrediction Distribution:")
    for label, count in pd.Series(predictions).value_counts().items():
        percentage = (count/len(predictions))*100
        print(f"{class_mapping[label]}: {count} images ({percentage:.2f}%)")
    
    # Print confidence statistics
    confidence_array = np.array(confidence_scores)
    print("\nConfidence Statistics:")
    print(f"Mean confidence: {confidence_array.mean():.4f}")
    print(f"Min confidence: {confidence_array.min():.4f}")
    print(f"Max confidence: {confidence_array.max():.4f}")
    
    # Save predictions
    submission_df.to_csv('swin_submissionFinal.csv', index=False)
    print("\nPredictions saved to 'swin_submission.csv'")
    
    return submission_df, confidence_scores

# Run predictions
from tqdm import tqdm
submission_df, confidence_scores = predict_test_set()

# Visualize confidence distribution
plt.figure(figsize=(10, 6))
plt.hist(confidence_scores, bins=50, edgecolor='black')
plt.title('Distribution of Prediction Confidence')
plt.xlabel('Confidence Score')
plt.ylabel('Count')
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
def visualize_predictions(n_samples=5):
    # Load some test images
    test_files = sorted(os.listdir('data/test'), 
                       key=lambda x: int(x.split('.')[0]))
    
    # Get same transform as used in prediction
    test_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                           std=[0.229, 0.224, 0.225])
    ])
    
    # Load model
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = SwinMaizeClassifier().to(device)
    checkpoint = torch.load('best_swin_model.pth')
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()
    
    plt.figure(figsize=(20, 4))
    
    with torch.no_grad():
        for i in range(n_samples):
            # Load and show original image
            img_path = os.path.join('data/test', test_files[i])
            img = Image.open(img_path).convert('RGB')
            
            # Get prediction
            img_tensor = test_transform(img).unsqueeze(0).to(device)
            outputs = model(img_tensor)
            probs = torch.softmax(outputs, dim=1)
            confidence, predicted = torch.max(probs, 1)
            
            # Plot
            plt.subplot(1, n_samples, i+1)
            plt.imshow(img)
            plt.title(f"Pred: {class_mapping[predicted.item()]}\nConf: {confidence.item():.4f}")
            plt.axis('off')
    
    plt.tight_layout()
    plt.show()

# Visualize some predictions
visualize_predictions(n_samples=5)

In [11]:
def tune_hyperparameters(model, train_loader, val_loader, n_trials=20):
    """
    Hyperparameter tuning using a grid search approach
    """
    # Define hyperparameter search space
    lr_space = [1e-5, 3e-5, 1e-4, 3e-4, 1e-3]
    weight_decay_space = [0.001, 0.01, 0.1]
    dropout_space = [0.1, 0.2, 0.3, 0.4, 0.5]
    batch_size_space = [8, 16, 32]
    
    best_val_acc = 0.0
    best_params = None
    device = next(model.parameters()).device
    
    # Initialize results tracking
    results = []
    
    for trial in range(n_trials):
        # Randomly sample hyperparameters
        lr = np.random.choice(lr_space)
        weight_decay = np.random.choice(weight_decay_space)
        dropout = np.random.choice(dropout_space)
        
        print(f"\nTrial {trial + 1}/{n_trials}")
        print(f"Parameters: lr={lr}, weight_decay={weight_decay}, dropout={dropout}")
        
        # Update model dropout
        for layer in model.modules():
            if isinstance(layer, nn.Dropout):
                layer.p = dropout
        
        # Initialize optimizer and scheduler
        optimizer = AdamW(
            model.parameters(),
            lr=lr,
            weight_decay=weight_decay,
            betas=(0.9, 0.999)
        )
        
        scheduler = CosineAnnealingLR(
            optimizer,
            T_max=5,  # Reduced epochs for tuning
            eta_min=1e-6
        )
        
        criterion = nn.CrossEntropyLoss()
        
        # Mini training loop for evaluation
        for epoch in range(5):  # Reduced epochs for tuning
            # Training phase
            model.train()
            running_loss = 0.0
            
            for batch_idx, (images, labels) in enumerate(train_loader):
                images, labels = images.to(device), labels.to(device)
                
                optimizer.zero_grad()
                outputs = model(images)
                loss = criterion(outputs, labels)
                
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                
                optimizer.step()
                
                running_loss += loss.item()
                
                if batch_idx % 50 == 0:
                    print(f'Epoch [{epoch+1}/5], Step [{batch_idx}/{len(train_loader)}], Loss: {loss.item():.4f}')
            
            # Validation phase
            model.eval()
            correct = 0
            total = 0
            val_loss = 0.0
            
            with torch.no_grad():
                for images, labels in val_loader:
                    images, labels = images.to(device), labels.to(device)
                    outputs = model(images)
                    loss = criterion(outputs, labels)
                    val_loss += loss.item()
                    
                    _, predicted = torch.max(outputs.data, 1)
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()
            
            # Calculate metrics
            epoch_loss = running_loss / len(train_loader)
            epoch_val_loss = val_loss / len(val_loader)
            val_accuracy = 100 * correct / total
            
            print(f'\nEpoch [{epoch+1}/5]:')
            print(f'Training Loss: {epoch_loss:.4f}')
            print(f'Validation Loss: {epoch_val_loss:.4f}')
            print(f'Validation Accuracy: {val_accuracy:.2f}%')
            
            scheduler.step()
        
        # Store results
        results.append({
            'lr': lr,
            'weight_decay': weight_decay,
            'dropout': dropout,
            'val_accuracy': val_accuracy,
            'train_loss': epoch_loss,
            'val_loss': epoch_val_loss
        })
        
        # Update best parameters if needed
        if val_accuracy > best_val_acc:
            best_val_acc = val_accuracy
            best_params = {
                'lr': lr,
                'weight_decay': weight_decay,
                'dropout': dropout
            }
            
            # Save best model state during tuning
            torch.save({
                'trial': trial,
                'model_state_dict': model.state_dict(),
                'best_params': best_params,
                'val_accuracy': best_val_acc,
            }, 'best_tuned_model.pth')
    
    # Convert results to DataFrame for analysis
    results_df = pd.DataFrame(results)
    
    # Visualize hyperparameter effects
    plt.figure(figsize=(15, 5))
    
    # Learning rate vs Validation Accuracy
    plt.subplot(1, 3, 1)
    sns.boxplot(x='lr', y='val_accuracy', data=results_df)
    plt.title('Learning Rate vs Validation Accuracy')
    plt.xticks(rotation=45)
    
    # Weight decay vs Validation Accuracy
    plt.subplot(1, 3, 2)
    sns.boxplot(x='weight_decay', y='val_accuracy', data=results_df)
    plt.title('Weight Decay vs Validation Accuracy')
    
    # Dropout vs Validation Accuracy
    plt.subplot(1, 3, 3)
    sns.boxplot(x='dropout', y='val_accuracy', data=results_df)
    plt.title('Dropout vs Validation Accuracy')
    
    plt.tight_layout()
    plt.show()
    
    print("\nBest Hyperparameters:")
    print(f"Learning Rate: {best_params['lr']}")
    print(f"Weight Decay: {best_params['weight_decay']}")
    print(f"Dropout: {best_params['dropout']}")
    print(f"Best Validation Accuracy: {best_val_acc:.2f}%")
    
    return best_params, results_df

In [13]:
def run_hyperparameter_tuning():
    # Initialize model
    model = SwinMaizeClassifier()
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    
    # Run hyperparameter tuning
    best_params, results_df = tune_hyperparameters(model, train_loader, val_loader, n_trials=10)
    
    # Save tuning results
    results_df.to_csv('hyperparameter_tuning_results.csv', index=False)
    
    return best_params, results_df

In [ ]:
best_params, results_df = run_hyperparameter_tuning()